# Parabolic model problem
We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \varepsilon \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$
with a suitable advective field $b$ (the wind).

In [1]:
import netgen.gui
%gui tk
import tkinter
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry

We generate geometry $(-1,1)^2$, mesh (Dirichlet boundaries everywhere):

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")
u = fes.TrialFunction()
v = fes.TestFunction()

time = 0.0
dt = 0.001

bcs =  ('bottom', 'right', 'top', 'left')


We define the field $b$ (the wind) as $b(x,y) = (2y(1-x^2),-2x(1-y^2))$.

In [3]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind")

We define and assemble bilinear forms for convection-diffusion stiffness and mass matrix seperately.

Note that we choose a non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [4]:
a = BilinearForm(fes, symmetric=False)
a += SymbolicBFI (0.01*grad(u)*grad(v) + b*grad(u)*v)
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += SymbolicBFI (u*v)
m.Assemble()

We want to use a simple implicit Euler time stepping method, i.e.

$$
  (M + \Delta t A) u^{n+1} = M u^n + f^{n+1}
$$

or in an incremental form:

$$
  (M + \Delta t A) (u^{n+1} - u^n) = - \Delta t A u^n + f^{n+1}.
$$

The incremental form has the advantage that $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).

For the time stepping method we want to set up linear combinations of matrices.

(Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [5]:
mstar = m.mat.CreateMatrix()
print(mstar)

Row 0:   0: 0.00411523   4: 0.00205761   35: 0.00205761   97: -0.000411523   98: 6.85871e-05   99: -0.000411523   100: 6.85871e-05   115: -0.000205761   116: -1.01644e-19   601: 0.000137174
Row 1:   1: 0.00411523   11: 0.00205761   12: 0.00205761   101: -0.000411523   102: 6.85871e-05   103: -0.000411523   104: 6.85871e-05   157: -0.000205761   158: 1.01644e-19   606: 0.000137174
Row 2:   2: 0.00411523   19: 0.00205761   20: 0.00205761   105: -0.000411523   106: 6.85871e-05   107: -0.000411523   108: 6.85871e-05   203: -0.000205761   204: 1.01644e-19   611: 0.000137174
Row 3:   3: 0.00411523   27: 0.00205761   28: 0.00205761   109: -0.000411523   110: 6.85871e-05   111: -0.000411523   112: 6.85871e-05   249: -0.000205761   250: 1.01644e-19   616: 0.000137174
Row 4:   0: 0.00205761   4: 0.0164752   5: 0.00170601   35: 0.00456558   36: 0.00367203   51: 0.00447399   97: -0.000411523   98: -6.85871e-05   99: -0.000205761   100: -1.11808e-19   113: -0.000341202   114: 5.6867e-05   115: -0.0

To access the entries we use the vector of nonzero-entries:

In [6]:
print(mstar.AsVector())

 0.00411523
 0.00205761
 0.00205761
 -0.000411523
 6.85871e-05
 -0.000411523
 6.85871e-05
 -0.000205761
 -1.01644e-19
 0.000137174
 0.00411523
 0.00205761
 0.00205761
 -0.000411523
 6.85871e-05
 -0.000411523
 6.85871e-05
 -0.000205761
 1.01644e-19
 0.000137174
 0.00411523
 0.00205761
 0.00205761
 -0.000411523
 6.85871e-05
 -0.000411523
 6.85871e-05
 -0.000205761
 1.01644e-19
 0.000137174
 0.00411523
 0.00205761
 0.00205761
 -0.000411523
 6.85871e-05
 -0.000411523
 6.85871e-05
 -0.000205761
 1.01644e-19
 0.000137174
 0.00205761
 0.0164752
 0.00170601
 0.00456558
 0.00367203
 0.00447399
 -0.000411523
 -6.85871e-05
 -0.000205761
 -1.11808e-19
 -0.000341202
 5.6867e-05
 -0.000913116
 0.000152186
 -0.000734407
 0.000122401
 -0.000894797
 0.000149133
 -0.000170601
 -8.47033e-20
 -0.000250797
 1.28749e-19
 -0.000196602
 -1.01644e-19
 0.000137174
 0.000113734
 0.000131068
 0.000167198
 0.00170601
 0.0106272
 0.00199129
 0.00332231
 0.00360759
 -0.000341202
 -5.6867e-05
 -0.000170601
 -9.40207e

Using the vector we can build the linear combination of the matrices $A$ and $M$:

In [7]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [8]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += SymbolicLFI(gaussp*v)
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [9]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,"u")

In [10]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 9.9999999999998976762.2979999999998583.87999999999968378.3070000000008359.59700000000012


## Supplementary 0: VTK Output (exporting nice pictures)

In [11]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],names=["sol"],filename="vtk_example1",subdivision=3)
vtk.Do()

In [12]:
vtk = VTKOutput(ma=mesh,coefs=[gfu,grad(gfu)],names=["sol","gradsol"],filename="vtk_example2",subdivision=3)
vtk.Do()

In [14]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],names=["sol"],filename="vtk_example3",subdivision=3)
vtk.Do()

gfu.Set((1-y*y)*x)

time = 0
tstep = 1 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
i = 0
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    i += 1
    if (i%10 == 0):
        vtk.Do()
print("")


 1.0000000000000007760.235000000000000180.475000000000000370.51500000000000030.59500000000000040.75500000000000060.8350000000000006


## Supplementary 1: time-dependent r.h.s. data
Next, we want to consider time-dependent r.h.s. data $f=f(t)$:

To this end, we introduce a parameter t representing the time. 

A Parameter is a constant CoefficientFunction the value of which can be changed with the Set-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
Draw(gausspt,mesh,"ft")
time = 0.0
while time < 10 - 0.5 * dt:
    t.Set(time)
    Redraw(blocking=True)
    time += 1e-4

Accordingly we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += SymbolicLFI(gausspt*v)
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
Draw(gfu,mesh,"u")

In [ ]:
tstep = 20 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

## Supplementary 2: Time dependent boundary conditions

Now, we want to consider changing boundary conditions $u|_{\partial \Omega} = u_D(t)$ and set $f=0$.

We use the simple implicit Euler time stepping method again for this problem. Now, we directly use the non-incremental form:
$$
  (M + \Delta t A) u^{n+1} = M u^n
$$
We homogenize w.r.t. to the boundary conditions, i.e. we split $u^{n+1} = u^{n+1}_0 + u^{n+1}_D$ where $u^{n+1}_D$ is some function in the FESpace with the correct boundary condition and $u^{n+1}_0$ is the sought for function with zero boundary values:

$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1}_0 = M u^n - \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1}_D
$$


In [ ]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    #non-incremental form 
    res.data = m.mat * gfu.vec
    gfu.Set(uD,BND)
    res.data -= mstar * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

## Supplementary 3: Singly diagonally implicit Runge-Kutta methods

We consider more sophisticated time integration methods, SDIRK methods. To simplify presentation we set $f=0$.

SDIRK methods for the semi-discrete problem $\frac{d}{dt} u = M^{-1} F(u) = -M^{-1} \cdot A u$ are of the form:
$$
  u^{n+1} = u^n + \Delta t M^{-1} \sum_{i=0}^{s-1} b_i k_i
$$
with
$$
k_i = - A u_i \quad \text{ where $u_i$ is the solution to } \quad 
(M + a^\ast \Delta t A) u_i = M u^n - \Delta t \sum_{i=0}^{i-1} a_{ij} k_j, \quad i=0,..,s-1.
$$
The coefficients a,b and c are stored in the butcher tableau:
$$
\begin{array}{c|cccc}
c_0 & a_{00} & 0 & \ddots& \\
c_1 & a_{10} & a_{11} & 0 & \ddots \\
\vdots & \vdots & \vdots & \ddots & 0\\
c_{s-1} & a_{s-1,0} & a_{s-1,1} & \dots& a_{s-1,s-1} \\ \hline
 & b_{0} & b_1 & \dots& b_{s-1} \\
\end{array}
$$
For an SDIRK method we have $a^\ast = a_{ii},~i=0,..,s-1$.

We can use for example the 2 stage SDIRK (order 3) method
$$
\begin{array}{c|cc}
p & p & 0 \\
1-p & 1-2p & p \\ \hline
 & 1/2 & 1/2 \\
\end{array}
$$
with $p = \frac{3 - \sqrt{3}}{6}$.

In [ ]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1
    
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p
    
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

butchertab = sdirk2()    
    
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

We have to update the $M^\ast$ matrix and reset initial data

In [ ]:
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
invmass = m.mat.Inverse(freedofs=fes.FreeDofs())

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    Mu0.data = m.mat * gfu.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.Set(uD,BND)
        ui.data = gfu.vec
        rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        
        # compute k[i] from ui
        k[i].data = - a.mat * ui
        
    t_intermediate += dt
    t.Set(time+t_intermediate)
    
    # Adding up the ks:
    gfu.Set(uD,BND)
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0
        
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    
print("")
time+=t_intermediate            